# 1.列表推导

 你有一个list：bag = [1, 2, 3, 4, 5]

 现在你想让所有元素翻倍，让它看起来是这个样子：[2, 4, 6, 8, 10]
 
 大多初学者，根据之前语言的经验会大概这样来做


In [4]:
bag = [1, 2, 3, 4, 5]  
for i in range(len(bag)):  
    bag[i] = bag[i] * 2
print(bag)

[2, 4, 6, 8, 10]


但是有更好的方法：

In [6]:
bag = [1, 2, 3, 4, 5]  
bag =[ i*2 for i in bag]
print(bag)

[2, 4, 6, 8, 10]


# 2遍历列表 

如果x是一个列表，你可以对它的元素进行迭代。多数情况下你不需要各元素的索引，但如果你非要这样做，那就用enumerate函数。
它像下边的样子：

In [7]:
for i in bag:
    print(i)
for index, element in enumerate(bag):  
    print(index, element)

2
4
6
8
10
0 2
1 4
2 6
3 8
4 10


注意遍历删除的问题是编程语言都会出现的问题，java中会出现同步修改异常。应该使用

In [8]:
l = [1,2,3,4]
for i in l:
    if i != 4:
         l.remove(i)
print(l)   # 期望得到4  ，结果得到 2,4

for item in filter(lambda x:x !=4,l):
    print(item)

[2, 4]
2


# 3. 元素互换

如果你是从java或者C语言转到Python来，可能会习惯于这样：
``` java
   String a = 5 ; 
   String  b = 10;
   //交换 a 和 b
   tmp = a  ;
   a = b  ;
   b = tmp;
```
   

但Python提供了一个更自然更好的方法！


In [9]:
a = 5  
b = 10  
# 交换a 和 b
a, b = b, a
print(a,b)

10 5


# 4 初始化列表
假如你要一个是10个整数0的列表，你可能首先想到：
``` python
bag = []  
for _ in range(10):  
    bag.append(0)
```
其实python 提供了：

In [1]:
bag = [0] * 10 
print(bag)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
import random
random.random()

0.6837986306483314

# 5 切片  

In [32]:
import numpy as np
bag = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 
#提取前5个元素,省略第一个元素。表示从0开始。
print(bag[:5])
#提取后5个元素,-5表示倒数第五个元素，省略第二个元素表示到最后一个元素。
print(bag[-5:])
#间隔2个元素进行提取,从1开始，按间隔:2取列表
print(bag[1::2])
#翻转列表
print(bag[::-1])

arrs = np.reshape(range(9),(3,3))
print(arrs)
print('arrs[:,-1]')
# 第一个冒号：表示横轴取默认size迭代，这里为[0-2].表示取所有横轴。
# 第二个冒号： 表示纵轴为末尾，表示取末尾列
print(arrs[:,-1])
print('arrs[:,:-1]')
# 第一个冒号：表示横轴取默认size迭代，这里为[0-2].  表示取所有横轴。
# 第二个冒号：-1 表示纵轴为末尾-1 ，这里为 1 表示取第二列
print(arrs[:,:-1])
print('arrs[:,0]')
# 第一个冒号：表示横轴取默认size迭代，这里为[0-2].  表示取所有横轴。
# 第二个0 ，表示取第一列。
print(arrs[:,0])
print('arrs[:,1:]')
# 第一个冒号：表示横轴取默认size迭代，这里为[0-2].  表示取所有横轴。
# 第二个1：，表示从第二列开始，取所有列。
print(arrs[:,1:])

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9]
[1, 3, 5, 7, 9]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
arrs[:,-1]
[2 5 8]
arrs[:,:-1]
[[0 1]
 [3 4]
 [6 7]]
arrs[:,0]
[0 3 6]
arrs[:,1:]
[[1 2]
 [4 5]
 [7 8]]


# 实践 python 读取日志

 * 给定时间范围、日志路径 下载日志 <br/>
 * 处理日志  <br/>
     * 超大文件，用with处理  <br/>
     * 如果出现 ITEM 错误， 则获取用户指令option （使用容器，只保存最新10条数据）<br/>
     * 对option 进行记录 （option ，time） <br/>
     * 每完成一个日志，将结果保存在文本中。<br/>
 * 展示进度
 
   

In [11]:
import ftplib
import re


def connectFtp():
    ftp = ftplib.FTP('10.5.72.110')
    ftp.login('app_bkg', 'TGBDdfh852')
    return ftp


def getFirstFile(ftp):
    ftp.cwd('/opt/applog/jcf/BKG_ORServer')
    files = ftp.nlst()  # 获取当前目录下所有文件名
    files = list(filter(lambda x: x.endswith('log.gz') == False, files))
    filename = files.pop(0)
    return filename


def download(ftp, filename):
    with open(filename, 'wb+') as fp:
        try:
            ftp.retrbinary("RETR %s" % filename, fp.write)
            fp.flush()
        except ftplib.error_perm:
            return False
        return True



def checkRegex(filename,cur_cmd,error_cmd_map):
    with open(filename) as f:
        for line in f:
            if re.match('.+INFO INPUT CMD.+', line) != None:
                cur_cmd[0]= re.findall('CMD:.*', line).pop()
          #  if re.match('.+ITEM.+', line) != None:
            if re.match('.+TRANSACTION TOTAL TIME.+', line) != None:
                count = error_cmd_map.get(cur_cmd[0])
                count = 1 if count == None else count+1
                error_cmd_map[cur_cmd[0]]=count
# ftp = connectFtp()
# filename = getFirstFile(ftp)
# print(filename)
# download(ftp, filename)
cur_cmd = [0]
error_cmd_map = {}
filename = 'log.2017-06-14.info.001.log'
checkRegex(filename,cur_cmd,error_cmd_map)
print(error_cmd_map)

ReadError: file could not be opened successfully